TOPIC: Data Warehousing Fundamentals
   1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.
   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.
   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.


In [ ]:
-- Create Products dimension table
CREATE TABLE Products (
  product_id SERIAL PRIMARY KEY,
  product_name VARCHAR(255) NOT NULL,
  category VARCHAR(255) NOT NULL,
  brand VARCHAR(255) NOT NULL
);

-- Create Customers dimension table
CREATE TABLE Customers (
  customer_id SERIAL PRIMARY KEY,
  customer_name VARCHAR(255) NOT NULL,
  address VARCHAR(255) NOT NULL,
  email VARCHAR(255) NOT NULL
);

-- Create Time dimension table
CREATE TABLE Time (
  date_id SERIAL PRIMARY KEY,
  date DATE NOT NULL,
  year INT NOT NULL,
  month INT NOT NULL,
  day INT NOT NULL,
  quarter INT NOT NULL,
  fiscal_year INT NOT NULL
);

-- Create Sales fact table
CREATE TABLE Sales (
  sales_id SERIAL PRIMARY KEY,
  product_id INT REFERENCES Products(product_id),
  customer_id INT REFERENCES Customers(customer_id),
  date_id INT REFERENCES Time(date_id),
  sales_amount DECIMAL(10, 2) NOT NULL
);


In [ ]:
-- Insert sample data into the Sales fact table
INSERT INTO Sales (product_id, customer_id, date_id, sales_amount)
VALUES
  (1, 1, 1, 100.00),
  (2, 2, 1, 150.00),
  (3, 3, 2, 200.00),
  (1, 2, 3, 120.00),
  (2, 1, 4, 180.00),
  (3, 3, 4, 220.00);


In [ ]:
SELECT p.product_name,SUM(s.sales_amount) AS total_sales_amount FROM Products p
JOIN Sales s ON p.product_id = s.product_id GROUP BY p.product_name;


In [ ]:
SELECT c.customer_name,SUM(s.sales_amount) AS total_sales_amount FROM Customers c
JOIN Sales s ON c.customer_id = s.customer_id GROUP BY c.customer_name;

In [ ]:
SELECT t.year,t.month,SUM(s.sales_amount) AS total_sales_amount FROM Time t
JOIN Sales s ON t.date_id = s.date_id WHERE t.year = 2022 GROUP BY t.year, t.month;


In [ ]:
SELECTp.product_name,c.customer_name,t.date,s.sales_amount FROM Sales s
JOIN Products p ON s.product_id = p.product_id 
JOIN Customers c ON s.customer_id = c.customer_id
JOIN Time t ON s.date_id = t.date_id WHERE p.product_name = 'Product A' AND c.customer_name = 'John Doe' AND t.date = '2022-07-01';


TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.


In [ ]:
import pandas as pd

# Extraction
source_files = ['file1.csv', 'file2.csv']
dataframes = []
for file in source_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Transformation
transformed_data = pd.concat(dataframes)
transformed_data['total_sales'] = transformed_data['quantity'] * transformed_data['unit_price']

# Loading
data_warehouse_connection = your_data_warehouse_connection
transformed_data.to_sql('sales_data', data_warehouse_connection, if_exists='replace', index=False)


TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.


In [ ]:
-- Create Students dimension table
CREATE TABLE Students (
  student_id SERIAL PRIMARY KEY,
  student_name VARCHAR(255) NOT NULL,
  student_major VARCHAR(255) NOT NULL,
  student_level VARCHAR(255) NOT NULL
);

-- Create Courses dimension table
CREATE TABLE Courses (
  course_id SERIAL PRIMARY KEY,
  course_name VARCHAR(255) NOT NULL,
  course_department VARCHAR(255) NOT NULL,
  course_credits INT NOT NULL
);

-- Create Time dimension table
CREATE TABLE Time (
  date_id SERIAL PRIMARY KEY,
  date DATE NOT NULL,
  year INT NOT NULL,
  month INT NOT NULL,
  day INT NOT NULL,
  semester VARCHAR(255) NOT NULL
);

-- Create Enrollments fact table
CREATE TABLE Enrollments (
  enrollment_id SERIAL PRIMARY KEY,
  student_id INT REFERENCES Students(student_id),
  course_id INT REFERENCES Courses(course_id),
  date_id INT REFERENCES Time(date_id),
  grade VARCHAR(2) NOT NULL
);


In [ ]:
SELECT c.course_name,COUNT(e.enrollment_id) AS total_enrollments FROM Courses c
JOIN Enrollments e ON c.course_id = e.course_id GROUP BY c.course_name;


In [ ]:
SELECT s.student_name, AVG(CAST(e.grade AS INTEGER)) AS average_grade FROM Students s
JOIN Enrollments e ON s.student_id = e.student_id GROUP BY s.student_name;


In [ ]:
SELECT t.semester,COUNT(e.enrollment_id) AS total_enrollments FROM Time t
JOIN Enrollments e ON t.date_id = e.date_id GROUP BY t.semester;


In [ ]:
SELECT s.student_name,c.course_name,e.grade FROM Enrollments e
JOIN Students s ON e.student_id = s.student_id
JOIN Courses c ON e.course_id = c.course_id WHERE e.enrollment_id = 1;


TOPIC: Performance Optimization and Querying
    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:
Utilize batch processing techniques to load data in bulk instead of individual row insertion.
      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.


In [ ]:
import time
import threading
from multiprocessing import Pool
import psycopg2

# Function to load data into the data warehouse
def load_data(data):
    # Connect to the data warehouse
    conn = psycopg2.connect(host="localhost", port="5432", database="your_database", user="your_username", password="your_password")
    cursor = conn.cursor()

    # Prepare the data for insertion (example: assume data is a list of tuples)
    values = ','.join(cursor.mogrify('(%s,%s,%s)', row).decode() for row in data)

    # Bulk insert the data
    insert_query = f"INSERT INTO your_table (col1, col2, col3) VALUES {values};"
    cursor.execute(insert_query)
    conn.commit()

    # Close the database connection
    cursor.close()
    conn.close()

# Function to measure the time taken for data loading
def measure_time_taken(data):
    start_time = time.time()

    # Approach 2: Multiprocessing
    pool = Pool()
    pool.map(load_data, data, chunksize=batch_size)
    pool.close()
    pool.join()

    end_time = time.time()

    time_taken = end_time - start_time
    print(f"Time taken: {time_taken} seconds")

# Define your data and batch size
data = [(1, 'John', 25), (2, 'Jane', 30), (3, 'David', 28)]  # Example data
batch_size = 1000  # Adjust the batch size according to your data size and performance requirements

# Measure the time taken to load data before optimization
measure_time_taken(data)

# Measure the time taken to load data after optimization
measure_time_taken(data)
